# Pull ITS_LIVE velocities along a glacier centerline

_modified by Aman KC, based on the version provided by [Jukes Liu](https://github.com/jukesliu)

Steps before you get started:

    
1. Clone the its_live repository using GitHub Desktop
Open GitHub Desktop.

Click "File" > "Clone Repository..."

Paste the URL: https://github.com/nasa-jpl/its_live

Choose a local path and click "Clone"

2. Open Terminal in the Cloned Repo Directory
Once cloned:

In GitHub Desktop, click the Repository menu > Open in Terminal (or right-click repo name and choose Open in Terminal).

This opens a terminal already pointed at your cloned its_live directory.

3. Create and Activate the itslive-notebooks Conda Environment
In the terminal that opens:

cd notebooks

conda env create -f environment.yml

The first command creates the environment from the environment.yml file in the repo (if present). If it's named differently, adjust accordingly.
Then run the following command in the terminal:

conda activate itslive-notebooks

4. Run or Modify the Code
In the terminal, type "_upyter lab". 
It will launch the notebook with the activated environment.

In [4]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from datetime import datetime
import xarray
from ordered_set import OrderedSet # pip install ordered-set
from pyproj import Proj # to get UTM coordinates from lat, lon
import cmocean
import fiona
import shapefile
import shapely
from shapely.geometry import shape, LineString
import pyproj
import math
import geopandas as gpd


# import additioonal functions
from additional_functions import *
# intialize projections:

# import the velocity widget from ITS LIVE
from velocity_widget import ITSLIVE
velocity_widget = ITSLIVE()

NoneType: None



## get variable at one location (v, vx, vy, va, va_error)

In [ ]:
# if the coordinates are in WGS 84 (lon, lat)
lon,lat = 75.246204, 38.693705 #example point at Karayaylak Glacier in HMA
#print("UTM7N: ", UTMx, UTMy)
variable = 'v' # vx, vy, v_error
point_xy = lon, lat
point_xy

# if the coordinates are in coordinate system other than WGS 84

source_crs = pyproj.CRS("EPSG:4326")  # Latitude, Longitude (WGS84)

target_crs = pyproj.CRS("ESRI: 102025") # High mountain Asia coordinate system, Alasks = EPSG : '4326', Greenland = EPSG : '3413'

transform_func = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)

x,y = pd.read_csv('your_dataframe')

lon,lat = transform_func.transform(x,y)

In [ ]:
ins3xr, pt_variable, point_tilexy] = velocity_widget.get_timeseries(point_xy, EPSG, variable)

### plottingg velocity at a point

In [ ]:
# Plot by sat groups
fig, axs = plt.subplots(2,1,figsize=(10,10))
ax_idx = 0
for df in [optical_df, SAR_df]:
    groups = df.groupby('sat')
    for name, group in groups:
        axs[ax_idx].plot(group.mid_date, mytomd(group.v), marker='o', linestyle='', ms=4, label=name, alpha=0.5)
    axs[ax_idx].legend()
    axs[ax_idx].set_ylabel('Surface speed (m/d)')
    # axs[ax_idx].set_ylim(0, np.nanmax(mytomd(np.concatenate([optical_df.v,SAR_df.v])))+2)
    axs[ax_idx].grid()
    ax_idx+=1
    
plt.show()

### optional: run this, if you have a centerline as a shapefile

In [2]:
# cline_path = '/Users/amankc/Terminus_Ablation/Saqqarliup_Sermia/Centerline/'

# cline_path = '/Users/amankc/High-Mountain-Asia/'

# clinefile = 'Garmo-glacier-cl.shp'

# f = fiona.open(cline_path+clinefile)
# fcoords = []
# for feature in f:
#     fcoords.append(feature['geometry']['coordinates'])
# fcoords = fcoords[0]
# a = 10
# coords = [];
# while a < len(fcoords):
#     coords.append(fcoords[a])
#     a += 10

# 71.78861266431407, 39.020204854870904
# 71.78860008300386, 39.01988481879073

### optional: run this, if you want to generate evenly spaced points

In [ ]:

# shapefile_path = cline_path + clinefile

# with fiona.open(shapefile_path, 'r') as source:
#     first_feature = next(iter(source))

#     line = shape(first_feature['geometry'])

# even_dist = 500
# len_cline = first_feature['properties']['length_m']

# n = int(len_cline/even_dist) 


# distances = np.linspace(0, line.length, n)


# points = []

# for distance in distances:
#     point = line.interpolate(distance)
#     points.append(point)


# fcoords = []

# for point in points:
#     x = point.x
#     y = point.y
#     fcoords.append((x, y))


# for i, (x, y) in enumerate(fcoords):
#     print(f"Point {i+1}: ({x:.3f}, {y:.3f})")


In [ ]:
x,y = pd.read_csv('your_dataframe')

lon,lat = transform_func.transform(x,y) # dont need this, if in already lon, lat
 
fcoords = lon,lat

for coords in fcoords:
    point_xy = coords
    [ins3xr, pt_variable, point_tilexy] = velocity_widget.get_timeseries(point_xy, EPSG, variable)

    [mid_dates_SAR, var_SAR, d1_SAR, d2_SAR, sat_SAR] = grab_v_by_sensor(ins3xr, pt_variable, 
                                                                200, # max separation of 30 days
                                                                ['1A','1B']) # SAR sensors
    SAR_df = pd.DataFrame(list(zip(mid_dates_SAR, var_SAR, d1_SAR, d2_SAR, sat_SAR)),
                          columns=['mid_date','v','d1','d2','sat'])
    SAR_df.head()

    [mid_dates_opt, var_opt, d1_opt, d2_opt, sat_opt] = grab_v_by_sensor(ins3xr, pt_variable, 
                                                                200, # max separation of 30 days
                                                                ['8','9','2A','2B']) 
    optical_df = pd.DataFrame(list(zip(mid_dates_opt, var_opt, d1_opt, d2_opt, sat_opt)),
                          columns=['mid_date','v','d1','d2','sat'])
    optical_df.head()

    # Plot by sat groups
    fig, axs = plt.subplots(2,1,figsize=(10,10))
    ax_idx = 0
    for df in [optical_df, SAR_df]:
        groups = df.groupby('sat')
        for name, group in groups:
            axs[ax_idx].plot(group.mid_date, mytomd(group.v), marker='o', linestyle='', ms=4, label=name, alpha=0.5)
        axs[ax_idx].legend()
        axs[ax_idx].set_ylabel('Surface speed (m/d)')
        # axs[ax_idx].set_ylim(0, np.nanmax(mytomd(np.concatenate([optical_df.v,SAR_df.v])))+2)
        axs[ax_idx].grid()
        ax_idx+=1

    plt.show()

## create equally spaced points along the centerline

In [ ]:
cline_path = '/Users/amankc/High-Mountain-Asia/'
# clinefile = 'Cenline_Saqqarliup_Sermia.shp' # spaced 250 m apart
clinefile = 'Khurdopin-Glacier-cl.shp'
# open centerline shapefile (points) and grab lat,lon coordinates
f = fiona.open(cline_path+clinefile)
fcoords = []
for feature in f:
    fcoords.append(feature['geometry']['coordinates'])
fcoords = fcoords[0]
a = 10
coords = [];
while a < len(fcoords):
    coords.append(fcoords[a])
    a += 10


source_crs = pyproj.CRS("EPSG:4326")  # Latitude, Longitude (WGS84)
# target_crs = pyproj.CRS("EPSG:32643") # High mountain Asia coordinate system
target_crs = pyproj.CRS("ESRI: 102025")
transform_func = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)

In [ ]:
shapefile_path = cline_path + clinefile

with fiona.open(shapefile_path, 'r') as source:
    # Read the first feature in the shapefile (assuming it is a LineString)
    first_feature = next(iter(source))
    
    # Convert the GeoJSON-like geometry into a Shapely geometry
    line = shape(first_feature['geometry'])

even_dist = 2000
len_cline = first_feature['properties']['length_m']
# Choose how many points you want along the line
n = int(len_cline/even_dist) # Number of evenly spaced points

# Generate equally spaced distances along the line from 0 to the total length
distances = np.linspace(0, line.length, n)

# Create an empty list to hold the point geometries
points = []

# Loop through each distance and interpolate a point on the line
for distance in distances:
    point = line.interpolate(distance)
    points.append(point)

# Create a list to hold (x, y) coordinates of the interpolated points
fcoords = []

# Loop through the list of Shapely point geometries to extract coordinates
for point in points:
    x = point.x
    y = point.y
    fcoords.append((x, y))

## plot velocities along the centerline

In [ ]:
for coords in fcoords:
    point_xy = coords
    [ins3xr, pt_variable, point_tilexy] = velocity_widget.get_timeseries(point_xy, EPSG, variable)

    [mid_dates_SAR, var_SAR, d1_SAR, d2_SAR, sat_SAR] = grab_v_by_sensor(ins3xr, pt_variable, 
                                                                30, # max separation of 30 days
                                                                ['1A','1B']) # SAR sensors
    SAR_df = pd.DataFrame(list(zip(mid_dates_SAR, var_SAR, d1_SAR, d2_SAR, sat_SAR)),
                          columns=['mid_date','v','d1','d2','sat'])
    SAR_df.head()

    [mid_dates_opt, var_opt, d1_opt, d2_opt, sat_opt] = grab_v_by_sensor(ins3xr, pt_variable, 
                                                                30, # max separation of 30 days
                                                                ['8','9','2A','2B']) 
    optical_df = pd.DataFrame(list(zip(mid_dates_opt, var_opt, d1_opt, d2_opt, sat_opt)),
                          columns=['mid_date','v','d1','d2','sat'])
    optical_df.head()

    # Plot by sat groups
    fig, axs = plt.subplots(2,1,figsize=(10,10))
    ax_idx = 0
    for df in [optical_df, SAR_df]:
        groups = df.groupby('sat')
        for name, group in groups:
            axs[ax_idx].plot(group.mid_date, mytomd(group.v), marker='o', linestyle='', ms=4, label=name, alpha=0.5)
        axs[ax_idx].legend()
        axs[ax_idx].set_ylabel('Surface speed (m/d)')
        # axs[ax_idx].set_ylim([0,12])
        # axs[ax_idx].set_ylim(0, np.nanmax(mytomd(np.concatenate([optical_df.v,SAR_df.v])))+2)
        axs[ax_idx].grid()
        ax_idx+=1

    plt.show()

## 2016-2018 Landsat 8 velocities for maybe every-other point on a single graph

In [ ]:
colors = ['#df33af','#122ee4','#12e370','#c9cd76','#e4538b','#401bd3','#13dbb0','#99d537','#ed2932']

coords_updated = []
for i in range(len(fcoords)):
    if i % 2 == 1:  
        coords_updated.append(fcoords[i])

start_date = datetime.datetime(2016, 1, 1)
end_date = datetime.datetime(2018, 12, 31)

plt.figure(figsize=(12,6))

for a, coords in enumerate(coords_updated):
    point_xy = coords
    [ins3xr, pt_variable, point_tilexy] = velocity_widget.get_timeseries(point_xy, EPSG, variable)

    # Landsat 8 = sensor '8'
    [mid_dates_opt, var_opt, d1_opt, d2_opt, sat_opt] = grab_v_by_sensor(ins3xr, pt_variable, 
                                                                         60, ['8']) 
    optical_df = pd.DataFrame(list(zip(mid_dates_opt, var_opt, d1_opt, d2_opt, sat_opt)),
                              columns=['mid_date','v','d1','d2','sat'])

    # Filter to 2016–2018
    mask = (optical_df['mid_date'] >= start_date) & (optical_df['mid_date'] <= end_date)
    optical_df = optical_df[mask]

    # Plot
    plt.scatter(optical_df['mid_date'], mytomd(optical_df['v']), 
             marker='o', linestyle='-', alpha=0.7, 
             color=colors[a], label=f'Point {a+1}')
    print(colors[a % len(colors)])

plt.ylim([0, 12])
plt.xlim(start_date, end_date)
plt.ylabel('Surface speed (m/d)')
plt.xlabel('Date')
# plt.title('Landsat 8 Velocities (2016–2018)')
# plt.legend(loc='upper right', fontsize='small')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:

from scipy.interpolate import make_interp_spline

colors = ['#df33af', '#122ee4', '#12e370', '#c9cd76', '#e4538b',
          '#401bd3', '#13dbb0', '#99d537', '#ed2932']

coords_updated = [fcoords[i] for i in range(len(fcoords)) if i % 2 == 1]

start_date = datetime.datetime(2016, 1, 1)
end_date = datetime.datetime(2018, 12, 31)

plt.figure(figsize=(12, 6))

for a, coords in enumerate(coords_updated):
    point_xy = coords
    [ins3xr, pt_variable, point_tilexy] = velocity_widget.get_timeseries(point_xy, EPSG, variable)

    [mid_dates_opt, var_opt, d1_opt, d2_opt, sat_opt] = grab_v_by_sensor(
        ins3xr, pt_variable, 200, ['8'])  # Landsat 8 only

    optical_df = pd.DataFrame(list(zip(mid_dates_opt, var_opt, d1_opt, d2_opt, sat_opt)),
                              columns=['mid_date', 'v', 'd1', 'd2', 'sat'])

    optical_df['mid_date'] = pd.to_datetime(optical_df['mid_date'])

    # Filter to 2016–2018 and remove NaNs
    filtered_group = optical_df[
        (optical_df['mid_date'] >= start_date) &
        (optical_df['mid_date'] <= end_date) &
        (~pd.isna(optical_df['v']))
    ].copy()

    # Add ordinal date and melt speed columns
    filtered_group['date_num'] = filtered_group['mid_date'].map(datetime.datetime.toordinal)
    filtered_group['v_val'] = mytomd(filtered_group['v'])

    # Drop duplicate dates
    filtered_group = filtered_group.sort_values('date_num').drop_duplicates(subset='date_num')


    # Skip if too few points to fit
    if len(filtered_group) < 4:
        continue

    # Prepare spline
    dates_num = filtered_group['date_num'].values
    v_vals = filtered_group['v_val'].values

    try:
        spl = make_interp_spline(dates_num, v_vals, k=2)  # quadratic spline
        x_smooth = np.linspace(dates_num.min(), dates_num.max(), 500)
        y_smooth = spl(x_smooth)

        # Plot smoothed curve
        plt.plot([datetime.datetime.fromordinal(int(x)) for x in x_smooth],
                 y_smooth, color=colors[a % len(colors)], label=f'Point {a+1}')

        # Optional: also plot raw points
        # plt.scatter(filtered_group['mid_date'], v_vals, alpha=0.3, color=colors[a % len(colors)])
    
    except ValueError as e:
        print(f"Skipping Point {a+1} due to error: {e}")
        continue

plt.ylim([0, 12])
plt.xlim(start_date, end_date)
plt.ylabel('Surface speed (m/d)')
plt.xlabel('Date')
# plt.title('Landsat 8 Velocities (2016–2018) with Smoothed Spline Fits')
# plt.legend(loc='upper right', fontsize='small', ncol=2)
plt.grid(True, color='gray', linestyle='--', linewidth=0.7)
plt.tight_layout()
plt.show()
